<a href="https://colab.research.google.com/github/piotr-stefanski/fruit-and-vegetable-classification/blob/testing/Klasyfikacja_owcow_i_warzyw.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Przygotowywanie danych

### Pobieranie bazy danych

In [ ]:
#instalacja pakietu gdown
!pip install -U --no-cache-dir gdown --pre
#usuwanie i tworzenie pustego katalogu data
!rm -rf data && mkdir data
#pobieranie zip-a ze zdjęciami
!gdown 1xMkKocUrc02qBX6hb3EudQDS1BNPdhDN -O data.zip
#rozpakowywanie pliku data.zip do katalogu data
!unzip -q data.zip -d data

  Attempting uninstall: gdown
    Found existing installation: gdown 4.7.3
    Uninstalling gdown-4.7.3:
      Successfully uninstalled gdown-4.7.3
Downloading...
From (original): https://drive.google.com/uc?id=1xMkKocUrc02qBX6hb3EudQDS1BNPdhDN
From (redirected): https://drive.google.com/uc?id=1xMkKocUrc02qBX6hb3EudQDS1BNPdhDN&confirm=t&uuid=5f5b9bc3-41d7-451c-9bb8-567cce348c95
To: /content/data.zip
100% 59.7M/59.7M [00:00<00:00, 74.2MB/s]


### Importowanie niezbędnych bibliotek

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import PIL
import tensorflow as tf
import glob
import os
import random
import shutil

from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator

### Zmiana nazw zdjec

In [ ]:
def rename_images_in_subdirectories(parent_directory):
    subdirectories = [d for d in os.listdir(parent_directory) if os.path.isdir(os.path.join(parent_directory, d))]

    for subdirectory in subdirectories:
        subdirectory_path = os.path.join(parent_directory, subdirectory)

        files = os.listdir(subdirectory_path)

        image_number = 1

        for file_name in files:
            if file_name.lower().endswith(('.png', '.jpg', '.jpeg', '.gif')):
                new_name = f"{subdirectory}_{image_number}.jpg"

                current_path = os.path.join(subdirectory_path, file_name)
                new_path = os.path.join(subdirectory_path, new_name)

                os.rename(current_path, new_path)

                print(f"Zmieniono nazwę: {file_name} na {new_name}")

                image_number += 1

main_directory = 'data'

rename_images_in_subdirectories(main_directory)


Strumieniowane dane wyjściowe obcięte do 5000 ostatnich wierszy.
Zmieniono nazwę: IMG_9373.JPG na banana_946.jpg
Zmieniono nazwę: r_51_100.jpg na banana_947.jpg
Zmieniono nazwę: IMG_9269.JPG na banana_948.jpg
Zmieniono nazwę: 99_100.jpg na banana_949.jpg
Zmieniono nazwę: IMG_8955.JPG na banana_950.jpg
Zmieniono nazwę: IMG_8571.JPG na banana_951.jpg
Zmieniono nazwę: IMG_8994.JPG na banana_952.jpg
Zmieniono nazwę: IMG_9114.JPG na banana_953.jpg
Zmieniono nazwę: IMG_8857.JPG na banana_954.jpg
Zmieniono nazwę: IMG_9289.JPG na banana_955.jpg
Zmieniono nazwę: IMG_9065.JPG na banana_956.jpg
Zmieniono nazwę: 113_100.jpg na banana_957.jpg
Zmieniono nazwę: r_130_100.jpg na banana_958.jpg
Zmieniono nazwę: r_68_100.jpg na banana_959.jpg
Zmieniono nazwę: IMG_9350.JPG na banana_960.jpg
Zmieniono nazwę: IMG_8610.JPG na banana_961.jpg
Zmieniono nazwę: r_66_100.jpg na banana_962.jpg
Zmieniono nazwę: IMG_9036.JPG na banana_963.jpg
Zmieniono nazwę: IMG_9007.JPG na banana_964.jpg
Zmieniono nazwę: r_69_100

### Ograniczenie zdjęć

In [ ]:
def limit_number_of_images_in_subdirectories(parent_directory, limit=200):
    subdirectories = [d for d in os.listdir(parent_directory) if os.path.isdir(os.path.join(parent_directory, d))]

    for subdirectory in subdirectories:
        subdirectory_path = os.path.join(parent_directory, subdirectory)

        files = os.listdir(subdirectory_path)

        if len(files) > limit:
            files_to_remove = len(files) - limit
            files_to_remove = random.sample(files, files_to_remove)

            for file_name in files_to_remove:
                file_path = os.path.join(subdirectory_path, file_name)
                os.remove(file_path)
                print(f"Usunięto plik: {file_name} z katalogu: {subdirectory}")

main_directory = 'data'

limit_number_of_images_in_subdirectories(main_directory, limit=200)


Usunięto plik: banana_595.jpg z katalogu: banana
Usunięto plik: banana_1292.jpg z katalogu: banana
Usunięto plik: banana_142.jpg z katalogu: banana
Usunięto plik: banana_1144.jpg z katalogu: banana
Usunięto plik: banana_447.jpg z katalogu: banana
Usunięto plik: banana_111.jpg z katalogu: banana
Usunięto plik: banana_556.jpg z katalogu: banana
Usunięto plik: banana_553.jpg z katalogu: banana
Usunięto plik: banana_744.jpg z katalogu: banana
Usunięto plik: banana_356.jpg z katalogu: banana
Usunięto plik: banana_1265.jpg z katalogu: banana
Usunięto plik: banana_1215.jpg z katalogu: banana
Usunięto plik: banana_784.jpg z katalogu: banana
Usunięto plik: banana_441.jpg z katalogu: banana
Usunięto plik: banana_23.jpg z katalogu: banana
Usunięto plik: banana_406.jpg z katalogu: banana
Usunięto plik: banana_178.jpg z katalogu: banana
Usunięto plik: banana_850.jpg z katalogu: banana
Usunięto plik: banana_624.jpg z katalogu: banana
Usunięto plik: banana_830.jpg z katalogu: banana
Usunięto plik: ba

### Weryfikowanie rozkładu klas w bazie danych  

In [ ]:
apple_count = len(list(glob.glob('data/apple/*.jpg')))
print(f"apple count {apple_count}")
avocado_count = len(list(glob.glob('data/avocado/*.jpg')))
print(f"avocado count {avocado_count}")
banana_count = len(list(glob.glob('data/banana/*.jpg')))
print(f"banana count {banana_count}")
kiwi_count = len(list(glob.glob('data/kiwi/*.jpg')))
print(f"kiwi count {kiwi_count}")
mandarin_count = len(list(glob.glob('data/mandarin/*.jpg')))
print(f"mandarin count {mandarin_count}")
orange_count = len(list(glob.glob('data/orange/*.jpg')))
print(f"orange count {orange_count}")
pear_count = len(list(glob.glob('data/pear/*.jpg')))
print(f"pear count {pear_count}")
pepper_count = len(list(glob.glob('data/pepper/*.jpg')))
print(f"pepper count {pepper_count}")
potato_count = len(list(glob.glob('data/potato/*.jpg')))
print(f"potato count {potato_count}")
tomato_count = len(list(glob.glob('data/tomato/*.jpg')))
print(f"tomato count {tomato_count}")

apple count 200
avocado count 200
banana count 200
kiwi count 156
mandarin count 166
orange count 160
pear count 200
pepper count 200
potato count 200
tomato count 200


### Podział na dane Trenujące i Walidacyjne

In [ ]:
data_path = 'data/'
catalogs = os.listdir(data_path)

train_path = 'train/'
val_path = 'val/'
test_path = 'test/'
os.makedirs(train_path, exist_ok=True)
os.makedirs(val_path, exist_ok=True)
os.makedirs(test_path, exist_ok=True)

for catalog in catalogs:
    catalog_path = os.path.join(data_path, catalog)
    images = os.listdir(catalog_path)

    train_images, temp = train_test_split(images, test_size=0.3)
    test_images, val_images = train_test_split(temp, test_size=0.5)

    for img in train_images:
        source_path = os.path.join(catalog_path, img)
        dest_path = os.path.join(train_path, catalog, img)
        os.makedirs(os.path.dirname(dest_path), exist_ok=True)
        shutil.copyfile(source_path, dest_path)

    for img in val_images:
        source_path = os.path.join(catalog_path, img)
        dest_path = os.path.join(val_path, catalog, img)
        os.makedirs(os.path.dirname(dest_path), exist_ok=True)
        shutil.copyfile(source_path, dest_path)

    for img in test_images:
        source_path = os.path.join(catalog_path, img)
        dest_path = os.path.join(test_path, catalog, img)
        os.makedirs(os.path.dirname(dest_path), exist_ok=True)
        shutil.copyfile(source_path, dest_path)



### Liczebnosc zbioru trenujacego i walidacyjnego

In [ ]:
apple_val_count = len(list(glob.glob('val/apple/*.jpg')))
apple_train_count = len(list(glob.glob('train/apple/*.jpg')))
apple_test_count = len(list(glob.glob('test/apple/*.jpg')))
print(f"apple val {apple_val_count} apple train {apple_train_count} apple test {apple_test_count}")


apple val 30 apple train 140 apple test 30


# Trenowanie modelu sieci neuronowej

### Zmienne niezbędne do trenowania modelu

In [ ]:
batch_size = 32
class_count = 10

img_height = 64
img_width = 64

### Ładowanie bazy danych wykorzystywanej do trenowania modelu

In [ ]:
train_ds = tf.keras.utils.image_dataset_from_directory(
  'train',
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 1317 files belonging to 10 classes.


### Ładowanie bazy danych wykorzystywanej do testowania wydajności modelu podczas uczenia

In [ ]:
val_ds = tf.keras.utils.image_dataset_from_directory(
  'val',
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 283 files belonging to 10 classes.


### Ładowanie bazy danych wykorzystywanej do testowania wydajności modelu



In [ ]:
test_ds = tf.keras.utils.image_dataset_from_directory(
  'test',
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 282 files belonging to 10 classes.


### Wyświetlenie nazwy klas występujących w bazie danych

In [ ]:
class_names = train_ds.class_names
print(f'class names: {class_names}')

class names: ['apple', 'avocado', 'banana', 'kiwi', 'mandarin', 'orange', 'pear', 'pepper', 'potato', 'tomato']


Drobne ustawienia optymalizacyjne

In [ ]:
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=tf.data.AUTOTUNE)

### Trenowanie

In [ ]:
cnn= tf.keras.models.Sequential()
cnn.add(tf.keras.layers.Conv2D(filters=16, kernel_size=3, activation='relu', input_shape=[64,64,3]))
cnn.add(tf.keras.layers.Conv2D(filters=16, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
cnn.add(tf.keras.layers.Conv2D(filters=64, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.Conv2D(filters=64, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
cnn.add(tf.keras.layers.Flatten())
cnn.add(tf.keras.layers.Dense(units=256, activation='relu'))
cnn.add(tf.keras.layers.Dense(units=256, activation='relu'))
cnn.add(tf.keras.layers.Dropout(0.5))
cnn.add(tf.keras.layers.Dense(units=10, activation='softmax'))


In [ ]:
cnn.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
cnn.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_17 (Conv2D)          (None, 62, 62, 16)        448       
                                                                 
 conv2d_18 (Conv2D)          (None, 60, 60, 16)        2320      
                                                                 
 max_pooling2d_8 (MaxPoolin  (None, 30, 30, 16)        0         
 g2D)                                                            
                                                                 
 conv2d_19 (Conv2D)          (None, 28, 28, 64)        9280      
                                                                 
 conv2d_20 (Conv2D)          (None, 26, 26, 64)        36928     
                                                                 
 max_pooling2d_9 (MaxPoolin  (None, 13, 13, 64)        0         
 g2D)                                                 

In [ ]:
epochs=30
history = cnn.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)

Epoch 1/30
42/42 [==============================] - 3s 14ms/step - loss: 10.1694 - accuracy: 0.4753 - val_loss: 0.2984 - val_accuracy: 0.9152
Epoch 2/30
42/42 [==============================] - 0s 9ms/step - loss: 0.4185 - accuracy: 0.8679 - val_loss: 0.0955 - val_accuracy: 0.9788
Epoch 3/30
42/42 [==============================] - 0s 9ms/step - loss: 0.1236 - accuracy: 0.9598 - val_loss: 0.1073 - val_accuracy: 0.9576
Epoch 4/30
42/42 [==============================] - 0s 9ms/step - loss: 0.1061 - accuracy: 0.9719 - val_loss: 0.1490 - val_accuracy: 0.9576
Epoch 5/30
42/42 [==============================] - 0s 9ms/step - loss: 0.0673 - accuracy: 0.9719 - val_loss: 0.2028 - val_accuracy: 0.9647
Epoch 6/30
42/42 [==============================] - 0s 9ms/step - loss: 0.1503 - accuracy: 0.9643 - val_loss: 0.1331 - val_accuracy: 0.9647
Epoch 7/30
42/42 [==============================] - 0s 9ms/step - loss: 0.2463 - accuracy: 0.9332 - val_loss: 0.0604 - val_accuracy: 0.9859
Epoch 8/30
42/42 [

### Ewaluacja

In [ ]:
def evaluate_model(model, test_ds):
    predictions = model.predict(test_ds)

    y_true = np.concatenate([y for x, y in test_ds], axis=0)

    y_pred = np.argmax(predictions, axis=1)

    accuracy = accuracy_score(y_true, y_pred)
    print(f"Dokładność modelu: {accuracy}")

    print("Raport klasyfikacyjny:")
    print(classification_report(y_true, y_pred))

evaluate_model(cnn, test_ds)

9/9 [==============================] - 0s 3ms/step
Dokładność modelu: 0.11347517730496454
Raport klasyfikacyjny:
              precision    recall  f1-score   support

           0       0.14      0.13      0.14        30
           1       0.10      0.10      0.10        30
           2       0.10      0.10      0.10        30
           3       0.09      0.09      0.09        23
           4       0.12      0.12      0.12        25
           5       0.17      0.17      0.17        24
           6       0.13      0.13      0.13        30
           7       0.17      0.17      0.17        30
           8       0.03      0.03      0.03        30
           9       0.10      0.10      0.10        30

    accuracy                           0.11       282
   macro avg       0.11      0.11      0.11       282
weighted avg       0.11      0.11      0.11       282

